# RNN PLAY GENERATOR

In [3]:
%tensorflow_version 2.x
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


***Dataset***

In [4]:
path_to_file = tf.keras.utils.get_file('shakespare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1130496/1115394 [==============================] - 0s 0us/step


***Loading your own data***

In [5]:
from google.colab import files
path_to_file = list(files.upload().keys())[0]

IndexError: ignored

# Read contents of file
Let's look at the contents of file

In [6]:
# Read and then decode for py2 compat
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

# length of the text is the number of characters in it
print('length of text: {} characters'.format(len(text)))

length of text: 1115394 characters


In [7]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



# Encoding
We will encode the each unique character

In [8]:
vocab = sorted(set(text))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
def text_to_int(text): 
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)


In [9]:
#lets look at how part of our text is encoded
print("Text:", text[:13])
print("Encoded", text_to_int(text[:13]))

Text: First Citizen
Encoded [18 47 56 57 58  1 15 47 58 47 64 43 52]


Here we will make a function which converts numeric to text

In [10]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return '' .join(idx2char[ints])
print(int_to_text(text_as_int[:13]))

First Citizen


# Creating Training Examples

input: Hell | output: ello

In [11]:
seq_length = 100 # length of sequence for a training example
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

Next we can use the batch method to turn this stream of characters into batches of desired length

In [12]:
sequences = char_dataset.batch(seq_length+1, drop_remainder = True)

Now we need to use these sequences of length 101 and split them into input and output

In [15]:
def split_input_target(chunk):
  input_text = chunk[:-1] #hell
  target_text = chunk[1:] #ello
  return input_text, target_text #hell, ello

dataset = sequences.map(split_input_target) # we use map to apply the above function to every entery

In [16]:
for x, y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [20]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab) # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).


BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

# Building the Model
Now it is time to build the model.We will use an embedding layer a LSTM and one dense layer that contains a node for each unique character
in our training data.The dense layer will give us a probability distribution over all nodes.

In [23]:
def build_model(vocab_size, embedding_dim, rnn_uints, batch_size):
  model = tf.keras.Sequential([
      tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                batch_input_shape = [batch_size, None]),
      tf.keras.layers.LSTM(rnn_uints,
                           return_sequences = True,
                           stateful = True,
                           recurrent_initializer="glorot_uniform"),
      tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (64, None, 256)           16640     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dense (Dense)               (64, None, 65)            66625     
                                                                 
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


# Creating a Loss Function
Now we are actually going to create our own loss function for this problem.This is because our model will output a (64,sequence_length,65)
shaped tensor that represents the probability distribution of each character at each timestep for every sequence in the batch.
However,before we do that let's have a look at a sample input and the output from our untrained model. This is so we can understand what the
model is actually giving us.

In [24]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch) # ask our model for aprediction on our first batch of training dataset
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)") # print out the output shape

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [25]:
# We can see that the prdicition id an array of 64 arrays, one for each entry in batch
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[ 1.74478907e-03 -4.58745519e-03  6.40024990e-03 ...  2.28898716e-03
    7.98028253e-04  4.07404313e-03]
  [ 3.67813604e-03 -1.00730418e-03  3.12371738e-03 ...  2.28986563e-03
    1.22551853e-03 -3.06183100e-03]
  [ 5.97823784e-03  2.97526503e-03 -4.10111528e-03 ...  6.77584577e-03
   -2.34531309e-03 -5.44930575e-03]
  ...
  [ 6.37668697e-03 -2.16520904e-03 -1.62774324e-02 ...  1.40031800e-02
   -1.03834718e-02 -1.61315724e-02]
  [ 3.61903780e-03 -9.34661925e-03 -1.43750608e-02 ...  1.80853568e-02
   -1.19085778e-02 -9.22965258e-03]
  [-7.10853189e-03 -1.26878032e-02 -6.26306934e-03 ...  1.46964854e-02
   -1.41625786e-02 -6.27733115e-03]]

 [[ 4.17111395e-03 -9.21805622e-05 -5.58738783e-03 ...  2.10063974e-03
    5.48746786e-04 -3.90784815e-03]
  [ 4.31536045e-03  1.03293941e-03 -6.50820229e-03 ...  3.47453984e-03
    1.51197496e-03 -9.01971012e-03]
  [ 3.51817533e-03 -6.95744436e-03 -6.07289979e-03 ...  1.06371902e-02
   -1.57277170e-03 -1.79312238e-03]
  ...
  [-1.037

In [26]:
# lets examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# norice this is a 2d array of length 100, where each interior array is the prediction for the next character at each time step

100
tf.Tensor(
[[ 0.00174479 -0.00458746  0.00640025 ...  0.00228899  0.00079803
   0.00407404]
 [ 0.00367814 -0.0010073   0.00312372 ...  0.00228987  0.00122552
  -0.00306183]
 [ 0.00597824  0.00297527 -0.00410112 ...  0.00677585 -0.00234531
  -0.00544931]
 ...
 [ 0.00637669 -0.00216521 -0.01627743 ...  0.01400318 -0.01038347
  -0.01613157]
 [ 0.00361904 -0.00934662 -0.01437506 ...  0.01808536 -0.01190858
  -0.00922965]
 [-0.00710853 -0.0126878  -0.00626307 ...  0.01469649 -0.01416258
  -0.00627733]], shape=(100, 65), dtype=float32)


In [28]:
# and finally we will look at prediction at hte first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# and of course its 65 values representing the probability of each character occuring next

65
tf.Tensor(
[ 1.7447891e-03 -4.5874552e-03  6.4002499e-03  7.0408703e-04
 -6.8087429e-03 -3.7496141e-04 -1.4836577e-03  1.6274641e-03
 -4.3734605e-03  1.8359058e-03  2.0534098e-03  5.1760906e-03
  3.0685086e-03 -9.6623658e-04 -4.6553873e-03  4.3298760e-03
  3.5202678e-04  2.9715451e-03 -3.4012992e-03  5.7629519e-04
 -2.2375612e-03 -5.5704121e-03 -6.2991213e-04  9.2583662e-04
  1.5664682e-03  2.6586107e-03  3.8239122e-03 -8.1424462e-04
 -2.6969274e-04  3.8822478e-04  4.7388044e-03  1.5490240e-03
  3.9598988e-03 -4.3742321e-03  3.5265575e-03  1.1627902e-03
  2.2004331e-03  4.5973877e-03 -2.9012698e-04 -4.3895788e-04
 -2.6791238e-03  1.2445783e-03 -1.1732074e-03 -1.9317524e-03
  2.4093280e-03  1.8689549e-05 -7.2859796e-03  4.9752831e-03
 -6.0120495e-03  2.1072282e-03 -2.2722331e-03 -3.4277013e-03
  3.1049738e-03 -8.0353161e-04  4.3310127e-03 -1.3461699e-03
 -4.0467880e-03  1.0683239e-03  9.7972096e-04  1.0027885e-03
 -1.3316108e-05 -7.9112016e-03  2.2889872e-03  7.9802825e-04
  4.074043

In [29]:
# If we want to determine the predicted character we need to sample the output distribution
sampled_indices = tf.random.categorical(pred, num_samples=1)

# #now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)
predicted_chars # and this is what the model predicted for training sequence 1

",'\n:&bA izvpaCKMCtNpaohxiQyRsc&mBjGKeubBzJP;eRLbvXqBfBGpGeEe&iIGhiMUbtJn&DSMdjwXFhI?.,Es&M3kDYxjQI-,"

So now we need to create a loss function that can compare that output to the expected output and give us some numeric value representing
how close the two were.

In [31]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

# Compiling the Model
At this point we can think of our problem as a classification problem where the model predicts the probabillity of each unique letter coming
next.

In [32]:
model.compile(optimizer = 'adam', loss = loss)